spark

In [6]:
import json
import numpy as np
from hops import model
from hops.model import Metric

# Query Model Repository for best anomaly detection model

In [ ]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [7]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [8]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '-0.5246837139129639'}

# Create Model Serving of Exported Model

In [9]:
from hops import serving

In [10]:
MODEL_NAME

'ganAml'

In [11]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model ganAml ...
Serving for model ganAml successfully created

In [12]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

ganAml

In [13]:
# Get serving status
serving.get_status(MODEL_NAME)

'Stopped'

# Start Model Serving Server

In [14]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: ganAml...
Serving with name: ganAml successfully started

In [15]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [16]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [17]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [20]:
eval_td.show(5)

+------+--------------------+
|target|           embedding|
+------+--------------------+
|     1|[-0.5829833, -0.9...|
|     1|[-1.0027698, -0.1...|
|     1|[-0.49065784, 0.8...|
|     1|[-0.20831347, 0.0...|
|     1|[-0.78685, -0.722...|
+------+--------------------+
only showing top 5 rows

In [30]:
def model_server(model_name, input):
    data = {"signature_name": "serving_default", "inputs": [input]}
    return serving.make_inference_request(model_name, data)['outputs']

In [34]:
scored_df = eval_td.read().rdd.map(lambda x: (x.target,model_server(MODEL_NAME, np.array(x.embedding).tolist()))).toDF()

In [35]:
scored_df.show()

+---+------------+
| _1|          _2|
+---+------------+
|  1|   [8.59989]|
|  1|[7.83957243]|
|  1|[6.27353191]|
|  1|[5.49751902]|
|  1|[4.80958414]|
|  1|[1.98385191]|
|  1|[9.39169598]|
|  1|[3.65841889]|
|  1| [13.041048]|
|  1|[7.63058901]|
|  1|[8.31788445]|
|  1|   [4.57649]|
|  1|[6.39205217]|
|  1|[3.77176976]|
|  1|[5.49540186]|
|  1|[8.60316372]|
|  1|[4.10919619]|
|  1|[17.1318588]|
|  1|[4.76529789]|
|  1|[3.22205448]|
+---+------------+
only showing top 20 rows